In [13]:
include("../LiPoSID.jl")

using LinearAlgebra
function ⊗(A::AbstractMatrix, B::AbstractMatrix)
    return kron(A,B)
end

function LME_operator_symbolic(H, J)
    
    Id = I(2)   
    U = -im*(Id ⊗ H - transpose(H) ⊗ Id)
    D = sum( 2*transpose(j')⊗j-Id⊗(j'*j) - transpose(j)*transpose(j')⊗Id for j in J )/2 
    
    Lᴸᴹᴱ = U + D
    
    return Lᴸᴹᴱ
end

function LME_operator_from_DMD(A)

    M = [ 0  0   1  1
          1  im  0  0 
          1  -im  0  0
          0  0  -1  1 ]/2
   
    Lᴰᴹᴰ =   M * A * inv(M)
   
    return Lᴰᴹᴰ 
   
end


LME_operator_from_DMD (generic function with 1 method)

In [14]:
using SymPy
@syms s

using DynamicPolynomials

function Transfer_Function(A, b)
    inv(I(4)*s - A)*b
end

bₓ = [1, 0, 0, 1]

function SumCoeffs2(poly)
    p = expand(numerator(poly))
    degrees = range(0,convert(Int32,SymPy.degree(p, s)))
    obj = sum(p.coeff(s^i)^2 for i in degrees)
    return obj
end

function sympy_to_dynamicpoly(sympy_expr::Sym)
    # Extract variables from the SymPy expression
    vars = free_symbols(sympy_expr)

    # Convert SymPy variables to string and then to Symbol
    var_symbols = [Symbol(string(v)) for v in vars]

    # Create DynamicPolynomials variables using @polyvar macro
    @eval begin
        @polyvar $(var_symbols...)
    end

    # Map SymPy variables to DynamicPolynomials variables
    var_map = Dict(vars[i] => eval(var_symbols[i]) for i in 1:length(vars))

    # Substitute using subs and convert to DynamicPolynomial
    dynamic_poly_expr = SymPy.subs(sympy_expr, var_map)
    dynamic_poly = eval(Meta.parse(string(dynamic_poly_expr)))
    return dynamic_poly
end

sympy_to_dynamicpoly (generic function with 1 method)

In [15]:
ω = symbols("ω", real=true)
γ = symbols("γ", real=true)

Hᴸᴹᴱ = [ ω        0
         0        0   ]


Jᴸᴹᴱ = [ 0     γ
         0     0. + 0im  ]

Lᴸᴹᴱ = LME_operator_symbolic(Hᴸᴹᴱ, [Jᴸᴹᴱ])

M = [   0  0   1  1
        1  im  0  0 
        1 -im  0  0
        0  0  -1  1  ] / 2

Aᴸᴹᴱ = real.(inv(M) * Lᴸᴹᴱ * M)



4×4 Matrix{Sym{PyCall.PyObject}}:
 -0.5*γ^2    -1.0⋅ω         0        0
    1.0⋅ω  -0.5*γ^2         0        0
        0         0  -1.0*γ^2  1.0*γ^2
        0         0         0        0

In [16]:
Gᴸᴹᴱ = Transfer_Function(Aᴸᴹᴱ, bₓ)

4-element Vector{Sym{PyCall.PyObject}}:
 (1.0*s + 0.5*γ^2)/(1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)
             1.0*ω/(1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)
                                1.0*γ^2/(1.0*s^2 + 1.0*s*γ^2)
                                                          1/s

In [17]:

Aᴰᴹᴰ⁻ˢᴮ = [ 0  -25     0      0 
            25 -0.04   0      0
            0   0     -0.04   0.04
            0   0      0      0   ]

Lᴰᴹᴰ = LME_operator_from_DMD(Aᴰᴹᴰ⁻ˢᴮ)

M = [   0  0   1  1
        1  im  0  0 
        1 -im  0  0
        0  0  -1  1  ] / 2

@assert abs(LiPoSID.frobenius_norm2(inv(M) * Lᴰᴹᴰ * M - Aᴰᴹᴰ⁻ˢᴮ)) < 1e-9


Gᴸᴹᴱ = Transfer_Function(Aᴸᴹᴱ, bₓ)[1:3]

Gᴰᴹᴰ = Transfer_Function(Aᴰᴹᴰ⁻ˢᴮ, bₓ)[1:3]

ΔG = together.(Gᴸᴹᴱ - Gᴰᴹᴰ)

polys = numerator.(ΔG)

obj = sum(SumCoeffs2.(polys))

objective = sympy_to_dynamicpoly(obj)

best_solution, best_method = LiPoSID.sos_min_newton(objective) 


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
termination status: SLOW_PROGRESS
solution status: FEASIBLE_POINT
optimum = 0.20006359990026795
Found a locally optimal solution by Ipopt, giving an upper bound: 2500.99999744.
The relative optimality gap is: 99.992001%.
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
termination status: SLOW_PROGRESS


solution status: FEASIBLE_POINT
optimum = 0.20006312642441468
Global optimality certified with relative optimality gap 0.000046%!
No higher TS step of the TSSOS hierarchy!
val_p = [2500.9999974370003, 0.20006361603736877]


(PolyVar{true}[γ, ω] => [-0.1789139827151704, 24.99998206753842], "scaled_tssos")

In [23]:
function TF_objective(Aˢʸᵐᵇ, Aˢⁱᵈ, b)

    Gˢʸᵐᵇ = Transfer_Function(Aˢʸᵐᵇ, b)[1:3]

    Gˢⁱᵈ = Transfer_Function(Aˢⁱᵈ, b)[1:3]

    ΔG = (together.(Gˢʸᵐᵇ - Gˢⁱᵈ)).^2

    polys = numerator.(ΔG)

    obj = sum(SumCoeffs.(polys))

    objective = sympy_to_dynamicpoly(obj)

    return objective

end

TF_objective (generic function with 1 method)

In [24]:
Aˢⁱᵈ = [  0  -25     0      0 
          25 -0.09   0      0
          0   0     -0.09   0.09
          0   0      0      0    ]

objective = TF_objective(Aᴸᴹᴱ, Aˢⁱᵈ, bₓ)

best_solution, best_method = LiPoSID.sos_min_newton(objective) 

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = -0.003191028352062961
Global optimality certified with relative optimality gap 0.000002%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = -0.0031910830691570234
Global optimality certified with relative optimality gap 0.000003%!
No higher TS step of the TSSOS hierarchy!
val_p = [-0.0031910491752928927, -0.0031910491752933368]


(PolyVar{true}[γ, ω] => [0.2989164909633352, 24.09713390688877], "scaled_tssos")